# DuckDB Demo Code

This notebook serves as an introduction to DuckDB, using the Python API. In the event that you are running this notebook separately, you will want to perform the following setup steps.

## Setup Steps

I [highly recommend you set up a virtual environment](https://youtu.be/Th9Noj_078A). If you choose not to use a virtual environment, the demos will still work as expected, assuming there are no Python package version conflicts.

All of the requirements are in `code\requirements.txt`.

Links to where you can get all of the data are available in `data\DataLocations.txt`. For these demos, I store data in a folder called `duckdbdata\`. This folder is **not** in source control, so you will need to create it locally.

In [ ]:
%pip install -r requirements.txt

## Load Packages

In [ ]:
import duckdb
import pandas as pd
import polars as pl

## Modules and Connections

There are two ways to use DuckDB in Python: direct access via the `duckdb` module or access via a connection. If you are familiar with the way connections happen in .NET using, for example, ADO, you know that there is a multi-step process:

1. Open the connection.
2. Using the open connection, send one or more commands.
3. Close the connection.

DuckDB's Python API allows this, but it also allows us to perform direct access. For example, these two commands are, in many respects, the same.

In [ ]:
duckdb.sql("SELECT 42 as X").show()

In [ ]:
con = duckdb.connect()
con.sql("SELECT 42 as X").show()
con.close()

In that case, why create a connection? There are a few important reasons:

1. `duckdb.sql()` runs commands against an in-memory database, rather than against a specific file. No tables persist to disk.
2. Creating a connection **may** run commands against an in-memory database, or against a database on disk. In the example above, we used `duckdb.connect()`, which still uses an in-memory database. But we can include a filename as a parameter and write to disk.
3. The `duckdb.sql()` commands run against a **shared global database**, meaning that if you are running multiple processes that each use DuckDB in non-connection mode, they write to the same in-memory database. This might cause unexpected conflicts due to matching names.

In general, use connections. There may be specific circumstances in which the global `duckdb.sql()` module is necessary, but they are fairly uncommon.

You can also maintain a connection and auto-close it using the `with` keyword. This closure allows the connection to be open until the end of the `with` block, after which Python closes the connection automatically.

In [ ]:
with duckdb.connect("../duckdbdata/file.db") as con:
    con.sql("CREATE OR REPLACE TABLE test (i INTEGER)")
    con.sql("INSERT INTO test VALUES (42)")
    con.table("test").show()

## Working with Data

The dataset we will use is a dataset of parking tickets from the city of Chicago, covering nearly two decades. The data is available thanks to Daniel Hutmacher, who [converted an open dataset into a well-designed SQL Server database](https://sqlsunday.com/2022/12/05/new-demo-database/). I have extracted much of this data into a Parquet file, and we will use that for demos.

If you do not already have a `chicago.db` file, this will create a new, empty file.

In [ ]:
con = duckdb.connect("../duckdbdata/chicago.db")

Next, we need to load the data from the Chicago Parking Tickets parquet file. This file is approximately 2GB in size and is available from the link in `data\DataLocations.txt`. Be sure to put it in a directory called `/duckdbdata` at the same level as the `/code` directory, or change the command to match where you have downloaded the file.

This uses Pandas to read the Parquet file into a Pandas DataFrame. DuckDB natively supports Pandas and Polars DataFrames.

In [ ]:
df1 = pd.read_parquet("../duckdbdata/ChicagoParkingTickets.parquet")

We can load a new table called `ChicagoParkingTickets` from the DataFrame we have loaded.

As a side note, notice that `CREATE OR REPLACE TABLE` and `CREATE TABLE IF NOT EXISTS` are both valid syntax for DuckDB.

In [ ]:
con.execute("CREATE TABLE IF NOT EXISTS ChicagoParkingTickets AS SELECT * FROM df1")

Using the `show()` function on a table allows us to see data in the table. By default, we only pull in a few rows--which is good, as this table is pretty big!

In [ ]:
con.table("ChicagoParkingTickets").show()

The `DESCRIBE` keyword provides us important metadata for our table.

In [ ]:
con.sql("DESCRIBE ChicagoParkingTickets")

DuckDB is intended for analytics, as we can see with the `SUMMARIZE` keyword. This provides us a quick descriptive analysis of numeric and string data types. For numbers, we get a five-number summary, as well as approximate distinct count, mean, and actual count.

Note that the approximate unique count does **not** use HyperLogLog, so it can be a little bit off. Here, it's off by approximately 5%.

In [ ]:
con.sql("SUMMARIZE ChicagoParkingTickets")

Summarizing a large table can take a while, so we can perform this one time and write the results into a separate table.

In [ ]:
con.sql("CREATE OR REPLACE TABLE cpt_summary AS SELECT * FROM (SUMMARIZE ChicagoParkingTickets)")

DuckDB supports a wide variety of ANSI SQL statements and patterns itself after PostgreSQL. In general, if it's a common part of ANSI SQL, you'll likely find it here.

In [ ]:
con.sql("SELECT COUNT(*) FROM ChicagoParkingTickets")

In [ ]:
con.sql("SELECT * FROM cpt_summary")

## Common (and Uncommon) Syntax

This example uses CASE expressions and aggregate functions.

In [ ]:
con.sql("""
    SELECT
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    GROUP BY
            Police_District,
            CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    ORDER BY SUM(PaymentIsOutstanding) DESC
""")

### Pivoting and Unpivoting

DuckDB supports pivoting using the standard ANSI SQL `PIVOT` keyword, but it also has its (vast) simplification of the problem. Here, we pivot the `records` table on license plate state, getting the count of in-state tickets and out-of-state tickets by police district.

We can also make use of common table expressions.

In [ ]:
con.sql("""
    WITH records AS (
    SELECT
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    GROUP BY
            Police_District,
            CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    )
    PIVOT records
    ON License_Plate_State
    USING SUM(TicketsOutstanding)
    ORDER BY Police_District
""")

We can also pivot on a subset of our data. In this case, we look only at specific police districts, ignoring the rest of the data. Then, our pivot gets the count of tickets for in-state and out-of-state vehicles in each of those districts.

In [ ]:
con.sql("""
    WITH records AS (
    SELECT
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    GROUP BY
            Police_District,
            CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    )
    PIVOT records
    ON Police_District IN (1.0, 2.0, 3.0, 4.0)
    USING SUM(TicketsOutstanding)
""")

In [ ]:
con.sql("""    
    WITH records AS (
        SELECT
            Police_District,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    )
    PIVOT records
    ON Police_District IN (1.0, 2.0, 3.0, 4.0)
    USING SUM(PaymentIsOutstanding)
    ORDER BY License_Plate_State
""").show(max_rows=100)

A common problem with pivoting is when you need more than one aggregate in your pivot. DuckDB handles that easily and auto-generates column names to make it clear which output is which.

In [ ]:
con.sql("""    
    WITH records AS (
        SELECT
            Police_District,
            Officer_ID,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    )
    PIVOT records
    ON Police_District IN (1.0, 2.0, 3.0, 4.0)
    USING SUM(PaymentIsOutstanding), MAX(Officer_ID)
    ORDER BY License_Plate_State
""").show(max_rows=100)

Unpivoting is also possible using the `UNPIVOT` keyword. Like `PIVOT`, you can follow the ANSI SQL syntax if you so desire, but DuckDB includes a simplified version of the syntax.

### Specifying Columns and Aliases

We can also use the `COLUMNS()` function to specify a list of columns based on some criteria. In this case, we want to include all columns, excluding license plate state.

In [ ]:
con.sql("""    
    WITH records AS (
        SELECT
            Police_District,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    ),
    pivoted AS (
        PIVOT records
        ON Police_District IN (1.0, 2.0, 3.0, 4.0)
        USING SUM(PaymentIsOutstanding)
    )
    UNPIVOT pivoted
    ON COLUMNS(* EXCLUDE(License_Plate_State))
    INTO
        NAME Police_District
        VALUE PaymentIsOutstanding
    ORDER BY Police_District ASC
""").show(max_rows=100)

Something that may not be familiar to users of SQL Server or other relational database platforms is that you can alias a column in the `SELECT` clause and use that alias in other clauses, such as `WHERE`, `GROUP BY`, or `HAVING`.

You can **not** use these aliases in the `ON` clause of a join, however.

In [ ]:
con.sql("""
    SELECT
        Police_District AS pd,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    WHERE
        pd IN (1.0, 2.0, 3.0, 4.0)
    GROUP BY
        pd,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    ORDER BY TicketsOutstanding DESC
""")

The `COLUMNS()` function also allows you to pass in lambda expressions, explaining what the shape of the set of columns should look like. For example, in this query, we retrieve the police district as well as all columns that start with the letters "Per."

Note that you cannot use `COLUMNS()` in the GROUP BY clause, so if you need to aggregate, you need to specify all of the columns out.

In [ ]:
con.sql("""
    SELECT
        Police_District AS pd,
        COLUMNS(col -> col LIKE 'Per%')
    FROM ChicagoParkingTickets
    WHERE
        pd IN (1.0, 2.0, 3.0, 4.0)
    LIMIT 10
""")

### Trailing Commas and Function Chaining

DuckDB has two quality of life improvements over most implementations of ANSI SQL. The first is that, like Python, it is not choosy about trailing commas. This makes it easier to change the order of your columns without needing to add or remove commas, or deal with ugly preceding commas.

The other is the ability to chain scalar functions. Instead of `TRIM(REPLACE(UPPER(Sector), ' ', '_'))`, you can chain each function using dot notation.

In [ ]:
con.sql("""
    SELECT
        Sector,
        Sector.upper().replace(' ', '_').trim() AS Sector_upper,
    FROM ChicagoParkingTickets
    WHERE
        Police_District IN (1.0, 2.0, 3.0, 4.0)
    LIMIT 10
""")

### Joins and Lateral Operations

Now we will re-shape and extend our data to allow for joins.

In [ ]:
con.sql("""
    CREATE OR REPLACE TABLE LicensePlateState
    (
        State_Abbreviation VARCHAR,
        Region VARCHAR
    );
        
    INSERT INTO LicensePlateState
    SELECT DISTINCT
        License_Plate_State,
        CASE WHEN License_Plate_State IN ('IL') THEN 'In-State'
             WHEN License_Plate_State IN ('IN', 'WI', 'MI', 'MO', 'IA', 'KY') THEN 'Neighbors'
             WHEN License_Plate_State IN ('AB', 'BC', 'GU', 'MB', 'MX', 'NB', 'NF', 'NS', 'ON', 'PE', 'PQ', 'PR', 'QU', 'XX', 'YT', 'ZZ') THEN 'Out-Of-Country'
             ELSE 'In-Country'
        END AS Region
    FROM ChicagoParkingTickets;
""")

In [ ]:
con.sql("""
    CREATE OR REPLACE TABLE CommunityDetails
    (
        Community_Name VARCHAR,
        Sector VARCHAR,
        Side VARCHAR,
        Hardship_Index DOUBLE,
        Per_capita_income DOUBLE,
        Percent_unemployed DOUBLE,
        Percent_without_diploma DOUBLE,
        Percent_households_below_poverty DOUBLE,
    );
        
    INSERT INTO CommunityDetails
    SELECT DISTINCT
        Community_Name,
        Sector,
        Side,
        Hardship_Index,
        Per_capita_income,
        Percent_unemployed,
        Percent_without_diploma,
        Percent_households_below_poverty
    FROM ChicagoParkingTickets;
""")

DuckDB supports all of the classic ANSI SQL joins, `INNER`, `LEFT OUTER`, `RIGHT OUTER`, `FULL OUTER`, and `CROSS`.

In [ ]:
con.sql("""
    SELECT
        lps.Region,
        COUNT(*) AS NumberOfTickets,
        SUM(cpt.PaymentIsOutstanding) AS TotalOutstanding,
        CAST(100.0 * SUM(cpt.PaymentIsOutstanding) / COUNT(*) AS DECIMAL(5,2)) AS PctOutstanding
    FROM ChicagoParkingTickets AS cpt
        INNER JOIN LicensePlateState AS lps
            ON cpt.License_Plate_State = lps.State_Abbreviation
    GROUP BY
        lps.Region
    ORDER BY
        lps.Region;
""").show(max_rows=100)

You can start a query with the `FROM` clause and add your `SELECT` clause later. If you do not include a `SELECT` clause, DuckDB will add an implicit `SELECT *`.

In [ ]:
con.sql("""
    FROM LicensePlateState AS lps
    ORDER BY
        lps.State_Abbreviation;
""").show(max_rows=100)

If you do choose to put the `SELECT` clause in later, note that it comes after `FROM` and any joins, but before `WHERE`. DuckDB supports the standard SQL ordering otherwise: `WHERE`, then `GROUP BY`, `HAVING`, `ORDER BY`, and `LIMIT`.

In [ ]:
con.sql("""
    FROM ChicagoParkingTickets AS cpt
        INNER JOIN LicensePlateState AS lps
            ON cpt.License_Plate_State = lps.State_Abbreviation
    SELECT
        lps.Region,
        COUNT(*) AS NumberOfTickets,
        SUM(cpt.PaymentIsOutstanding) AS TotalOutstanding,
        CAST(100.0 * SUM(cpt.PaymentIsOutstanding) / COUNT(*) AS DECIMAL(5,2)) AS PctOutstanding
    WHERE
        lps.Region = 'In-State'
    GROUP BY
        lps.Region
    ORDER BY
        lps.Region;
""").show(max_rows=100)

The results, as you would expect, will be the same regardless of whether you put `FROM` first or `SELECT` first.

In [ ]:
con.sql("""
    SELECT
        lps.Region,
        COUNT(*) AS NumberOfTickets,
        SUM(cpt.PaymentIsOutstanding) AS TotalOutstanding,
        CAST(100.0 * SUM(cpt.PaymentIsOutstanding) / COUNT(*) AS DECIMAL(5,2)) AS PctOutstanding
    FROM ChicagoParkingTickets AS cpt
        INNER JOIN LicensePlateState AS lps
            ON cpt.License_Plate_State = lps.State_Abbreviation
    WHERE
        lps.Region = 'In-State'
    GROUP BY
        lps.Region
    ORDER BY
        lps.Region;
""").show(max_rows=100)

The ANSI SQL equivalent of T-SQL's `APPLY` operator is the `LATERAL` operator. DuckDB supports this.

The basic idea of the `LATERAL` operator is, for each element on the left-hand side, we execute the function or expression in parentheses and make that the right-hand side.

In this case, we build up a quick table with values ranging from 0-4 inclusive as our left-hand side, and call that column `i`. Then, for each value of `i`, we add 1 to it and call it `j`.

Similar to the `APPLY` operator in T-SQL, we don't explicitly join our two sets, but we do need a comma.

In [ ]:
con.sql("""
    SELECT *
    FROM
        range(5) t(i),
        LATERAL (SELECT i + 1) t2(j);
""")

The result of a `LATERAL` operator can be zero or more records. If we have multiple records, it's equivalent to a join with multiple matching rows on the right-hand side for a given record on the left-hand side.

We can also use `ORDER BY ALL` to order by each column, without needing to specify them.

In [ ]:
con.sql("""
    SELECT *
    FROM
        generate_series(0, 2) t(i),
        LATERAL (SELECT i + 10 UNION ALL SELECT i + 100) t2(j)
    ORDER BY ALL;
""")

Positional joins are a way of ensuring a one to one match between two tables. The concept here is that we want the first record from t1 to match the first record from t2, the second record from t1 to match the second record from t2, and so on. But we don't have any explicit join criteria.

In [ ]:
con.sql("""
    CREATE OR REPLACE TABLE t1 (x INTEGER);
    CREATE OR REPLACE TABLE t2 (s VARCHAR);

    INSERT INTO t1 VALUES (1), (2), (3);
    INSERT INTO t2 VALUES ('a'), ('b');

    SELECT *
    FROM t1
    POSITIONAL JOIN t2;
""")

We also can include tables in `LATERAL` operations, not just expressions. In this example, we retrieve the largest value of s from a given table based on matching our records CTE.

In [ ]:
con.sql("""
    WITH records AS
    (
        SELECT *
        FROM t1
        POSITIONAL JOIN t2
    )        
    SELECT *
    FROM range(5) t(i),
        LATERAL (SELECT s FROM records WHERE t.i = records.x ORDER BY s DESC LIMIT 1) AS t2(s);
""")

## Support for Polars

DuckDB's Python API has native support for Polars DataFrames, not just Pandas. Here, we create a DataFrame called `df` and query directly from it.

In [ ]:
df = pl.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "fruits": ["banana", "banana", "apple", "apple", "banana"],
        "B": [5, 4, 3, 2, 1],
        "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
    }
)
duckdb.sql("SELECT * FROM df").show()

We can also convert a DuckDB result set into a Polars (or Pandas) DataFrame, allowing us to use any functions or methods in either of those libraries on our data.

In [ ]:
df = duckdb.sql("""
    SELECT 1 AS id, 'banana' AS fruit
    UNION ALL
    SELECT 2, 'apple'
    UNION ALL
    SELECT 3, 'mango'"""
).pl()
print(df)

In [ ]:
type(df)

## Generating Query Plans

In case you want to understand the performance profile of a given query, DuckDB has the `explain()` function in the Python API, as well as the `EXPLAIN` keyword in the core language.

This generates a pretty printed or graphical output (depending on your interface) that shows step by step what the query engine is doing to build your result.

In [ ]:
# EXPLAIN plan
con.sql("""
    WITH records AS (
        SELECT
            Police_District,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    ),
    pivoted AS (
        PIVOT records
        ON Police_District IN (1.0, 2.0, 3.0, 4.0)
        USING SUM(PaymentIsOutstanding)
    )
    UNPIVOT pivoted
    ON COLUMNS(* EXCLUDE(License_Plate_State))
    INTO
        NAME Police_District
        VALUE PaymentIsOutstanding
    ORDER BY Police_District ASC
""").explain()

## Managing Data Deletion

Data management and deletion in DuckDB is a bit different from major database systems. In this case, we'll create a new copy of our Chicago Parking Tickets data and load it into the chicago.db file.

In [ ]:
con.execute("CREATE TABLE ChicagoParkingTickets_DELETEME AS SELECT * FROM df1")

Reviewing the data file on disk, we can see that the size has approximately doubled.

Now let's drop that table.

In [ ]:
con.execute("DROP TABLE ChicagoParkingTickets_DELETEME")

Note that this creates a write-ahead log (.wal) file rather than actually deleting data. The file size is still ~3.0 GB.

One way to clean up data in PostgreSQL is to run the vacuum command. Let's try that.

In [ ]:
con.execute("VACUUM")

Unfortunately, this does nothing and, in fact, `VACUUM` is essentially a no-op in DuckDB, there simply for compatibility purposes.

Instead, we want to use the `CHECKPOINT` command to flush any insert, update, or delete operations.

In [ ]:
con.execute("CHECKPOINT")

Even so, this still doesn't change the size of our file!

If file size is a concern (and it may not be, given the way we normally work with DuckDB), you can make a copy of the existing data to a new file. This will ignore any deleted tuples and only move active data.

The first step we'll perform is to make sure there is a file called chicago2.db.

In [ ]:
# Make sure the database exists
con2 = duckdb.connect("../duckdbdata/chicago2.db")
con2.close()

Next, we'll close the chicago.db connection.

In [ ]:
con.close()

In the global module, we'll attach chicago2 and chicago as db2 and db1, respectively. Then, we'll copy the active contents of db1 into db2.

This will take a little bit of time, but when it's done, db2 will be approximately 1.5 GB, exactly what we want!

In [ ]:
duckdb.execute("""
    ATTACH '../duckdbdata/chicago2.db' AS db2;
    ATTACH '../duckdbdata/chicago.db' AS db1;
    COPY FROM DATABASE db1 to db2;
""")

## DuckDB Extensions

Like PostgreSQL, DuckDB has a variety of extensions. The DuckDB team is responsible for some of these, but there are also community extensions that you can install. There are dozens of extensions available to us.

In [ ]:
duckdb.sql("SELECT * FROM duckdb_extensions()").show()

Installing an extension is very simple. After we install an extension, we can load it to make use of it.

In [ ]:
duckdb.sql("INSTALL spatial; LOAD spatial;")

In case you want to see which extensions you have installed, there's a function for that.

In [ ]:
duckdb.sql("SELECT * FROM duckdb_extensions() WHERE installed = true").show()

We can make use of the Spatial extension and take advantage of spatial queries.

In [ ]:
duckdb.sql("SELECT ST_AsGeoJSON('POLYGON((0 0, 0 1, 1 1, 1 0, 0 0))'::GEOMETRY);")

In [ ]:
duckdb.sql("""
    SELECT CAST({
        type: 'Feature', 
        geometry: ST_AsGeoJSON(ST_Point(1,2)), 
        properties: { 
            name: 'my_point' 
        } 
    } AS JSON) AS geojson;
""")

In [ ]:
duckdb.sql("SELECT st_distance('POINT(0 0)'::GEOMETRY, 'POINT(3 3.15)'::GEOMETRY);")

## SQL Injection

Like any other data platform, DuckDB is susceptible to SQL injection if you write injectable code.

In this example, we have two functions that write out the input `x`. The first function directly writes `x` without any type of sanitization. The second function uses a parameterized query.

In [ ]:
def run_duckdb_sql_injectable(x):
    """
    This function is an example of how to run a SQL query with an injectable parameter.
    It uses the DuckDB Python API to execute the SQL query and return the result.
    """
    con = duckdb.connect("../duckdbdata/chicago.db")
    result = con.execute(f"SELECT {x} as X").fetchall()
    con.close()
    return result

def run_duckdb_sql_parameter(x):
    """
    This function is an example of how to run a SQL query with a parameter.
    It uses the DuckDB Python API to execute the SQL query and return the result.
    """
    con = duckdb.connect("../duckdbdata/chicago.db")
    result = con.execute("SELECT ? as X", (x,)).fetchall()
    con.close()
    return result

If we run these two queries with a reasonable value, we get back a reasonable result.

In [ ]:
run_duckdb_sql_injectable(42)

In [ ]:
run_duckdb_sql_parameter(42)

But if I run the injectable query with an arbitrary SQL command, I can escape out of the query and run my own query, such as retrieving all of the DuckDB settings or even secrets!

In [ ]:
run_duckdb_sql_injectable("42; FROM duckdb_settings();--")

But with the parametrized query, we cannot escape the parameter, and thus our query is safe.

In [ ]:
run_duckdb_sql_parameter("42; FROM duckdb_settings();--")